In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from scipy.sparse import hstack

In [ ]:
# Read the dataset
train = pd.read_csv("/kaggle/input/llm-classification-finetuning/train.csv")
test = pd.read_csv("/kaggle/input/llm-classification-finetuning/test.csv")
submission = pd.read_csv("/kaggle/input/llm-classification-finetuning/sample_submission.csv")

In [ ]:
submission

In [ ]:
test.info()

In [ ]:
print(f"train data shape: {train.shape}")
print(f"test data shape: {test.shape}")
print(f"sample_submission data shape: {submission.shape}")

In [ ]:
print(test['prompt'][0])
print(test["response_a"][0])
print(test["response_b"][0])

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt


wordcloud = WordCloud(width=800, height=800).generate(' '.join(test['prompt'].dropna()))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
train.isnull()
test.isnull()
submission.isnull()

In [ ]:
#Check for duplicate data
print(test.duplicated().sum())
print(train.duplicated().sum())
print(submission.duplicated().sum())

In [ ]:
#check for duplicate id's
total_id = len(train["id"])
total_unique_id = len(train["id"].unique())

print("Total number of 'id' duplicates:")
print(total_id - total_unique_id)

In [ ]:
#barchart - model winner: winner model a, winner model b or winner tie
def which_winner(value):
    if  value["winner_model_a"] == 1:
         #winner model a
         value["winner_model_b"] = 0
         value["winner_tie"] = 0
         return 0
    elif value["winner_model_b"] == 1:
         #winner model b
         return 1
    elif value["winner_tie"] == 1:
         #winner tie
         return 2
    return None

train["winner"] = train.apply(which_winner, axis=1)

train["winner_model"] = train["winner"].astype(str)
train.loc[train["winner_model"] == "0", "winner_model"] = "model a"
train.loc[train["winner_model"] == "1", "winner_model"] = "model b"
train.loc[train["winner_model"] == "2", "winner_model"] = "winner tie"

result_model_winner = train["winner_model"].value_counts()
print("model winner:", result_model_winner)

print("-----")
print("Matplotlib barchart, model winner:")

barWidth = 0.75
plt.figure(figsize=(8, 7))
plt.bar(result_model_winner.index, result_model_winner.values, barWidth, color='b')
plt.ylabel('Counts', fontweight ='bold', fontsize = 15)
plt.xlabel('Model winner', fontweight ='bold', fontsize = 15)
plt.title('LLMs Value Counts - model winner', fontweight ='bold', fontsize = 15)
plt.show()

In [ ]:
#transform text data into numerical form
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features = 150) #without max_features it crashes due to memory limit
vectorizer_prompt = vectorizer.fit_transform(train["prompt"])
print(vectorizer.idf_)
print(vectorizer.get_feature_names_out())
vectorizer_response_a = vectorizer.fit_transform(train["response_a"])
print(vectorizer.idf_)
print(vectorizer.get_feature_names_out())
vectorizer_response_b = vectorizer.fit_transform(train["response_b"])
print(vectorizer.idf_)
print(vectorizer.get_feature_names_out())



temp_prompt = vectorizer.transform(train["prompt"])
temp_response_a = vectorizer.fit_transform(train["response_a"])
temp_response_b = vectorizer.fit_transform(train["response_b"])

print("vectorizer representation for 'prompt':\n", temp_prompt.toarray())
print("vectorizer representation for 'response a':\n", temp_response_a.toarray())
print("vectorizer representation for 'response b':\n", temp_response_b.toarray())

print("Number of elements for the vectorizer representation for 'prompt':\n", temp_prompt.shape)
print("Number of elements for the vectorizer representation for 'response a':\n", temp_response_a.shape)
print("Number of elements for the vectorizer representation for 'response b':\n", temp_response_b.shape)

In [ ]:
#Using a simple feature and label selection
import numpy as np

# the prediction target
train_y = train["winner"].values
#choosing "features"
train_X = np.concatenate((temp_prompt.toarray(), temp_response_a.toarray(), temp_response_b.toarray()), axis=1)

print("Selecting The Prediction Target and Choosing Features Complete")

In [ ]:
from sklearn.linear_model import LogisticRegression
from datetime import datetime

#start time to calculate the execution time
start_time = datetime.now()

print("Use logistic regression")
#Apply the Logistic Regression
model = LogisticRegression(max_iter=500, multi_class='multinomial', solver='saga')
model.fit(train_X, train_y)
#end time
end_time = datetime.now()
#calculate the execution time
execution_time = (end_time - start_time).total_seconds()
print(f"The execution time is : {execution_time} secondes")

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Split into validation and training data
train_X_train, train_X_val, train_y_train, train_y_val = train_test_split(
    train_X, train_y, test_size=0.2, random_state=42
)

# Record start time to calculate the execution time
start = datetime.now()

# Make predictions on the validation set
value_y_predict = model.predict(train_X_val)
print('Model predicted values:', value_y_predict)
print('True values:', train_y_val)

# Predicted probabilities
value_y_probabilities = model.predict_proba(train_X_val)
print('Model prediction probabilities (class-wise):\n', value_y_probabilities)

# Model accuracy
score = model.score(train_X_val, train_y_val)
print('The Model Accuracy Score:', score)

# Confusion matrix
conf_matrix = confusion_matrix(train_y_val, value_y_predict)
print("Confusion Matrix:\n", conf_matrix)

# Classification report
report = classification_report(train_y_val, value_y_predict)  # Arguments fixed
print("Classification Report:\n", report)

In [ ]:
from sklearn.metrics import log_loss

model_log_loss = log_loss(train_y_val, value_y_probabilities)

print('Model Log loss:', model_log_loss)

In [ ]:
vectorizer = TfidfVectorizer(max_features = 150) #without max_features it crashes due to memory limit
vectorizer_prompt = vectorizer.fit_transform(test["prompt"])
print(vectorizer.idf_)
print(vectorizer.get_feature_names_out())
vectorizer_test_response_a = vectorizer.fit_transform(test["response_a"])
print(vectorizer.idf_)
print(vectorizer.get_feature_names_out())
vectorizer_test_response_b = vectorizer.fit_transform(test["response_b"])
print(vectorizer.idf_)
print(vectorizer.get_feature_names_out())
temp_test_prompt = vectorizer.transform(test["prompt"])
temp_test_response_a = vectorizer.fit_transform(test["response_a"])
temp_test_response_b = vectorizer.fit_transform(test["response_b"])
print("vectorizer representation for 'prompt':\n", temp_test_prompt.toarray())
print("vectorizer representation for 'response a':\n", temp_test_response_a.toarray())
print("vectorizer representation for 'response b':\n", temp_test_response_b.toarray())

print("Number of elements for the vectorizer representation for 'prompt':\n", temp_test_prompt.shape)
print("Number of elements for the vectorizer representation for 'response a':\n", temp_test_response_a.shape)
print("Number of elements for the vectorizer representation for 'response b':\n", temp_test_response_b.shape)
test_X = np.concatenate((temp_test_prompt.toarray(), temp_test_response_a.toarray(), temp_test_response_b.toarray()), axis=1)
value_test_y_probabilities = model.predict_proba(test_X)
print('Model winner prediction, probability', value_test_y_probabilities) #winner model a | winner
output = pd.DataFrame({'id': test.id,
                        'winner_model_a': value_test_y_probabilities[:, 0],
                        'winner_model_b': value_test_y_probabilities[:, 1],
                        'winner_tie': value_test_y_probabilities[:, 2]})
output.to_csv('submission.csv', index=False)